<a href="https://colab.research.google.com/github/raheelam98/langgraph_guru/blob/main/rag_from_scratch/rag_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Leaderboard

1.   [Arena](https://lmarena.ai/)
2.   [Berkeley Function-Calling Leaderboard](https://gorilla.cs.berkeley.edu/leaderboard.html)
3. sora video

4.   [huggingface](https://huggingface.co/spaces/ArtificialAnalysis/Video-Generation-Arena-Leaderboard)



[Making it easier to build human-in-the-loop agents with interrupt](https://blog.langchain.dev/making-it-easier-to-build-human-in-the-loop-agents-with-interrupt/)

**Google Generative AI Embeddings**

In [ ]:
# Install the required packages:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "rag_ai_agent"

In [ ]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# Initialize the ChatGoogleGenerativeAI with the Gemini model

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model to use
    max_retries=2,
    api_key=gemini_api_key    # Provide the Google API key for authentication
)

In [ ]:
# Invoke the LLM with a query
result = llm.invoke("hi")
result

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-95a66a05-1b9a-46fc-ac9e-604d12249c6b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_core.vectorstores import InMemoryVectorStore

# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001" , # Specify the embedding model to use
#     api_key=gemini_api_key
# )#vector = embeddings.embed_query("hello, world!")
# # vector[:5]

# embeddings.embed_query("Hello")


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=gemini_api_key)

vector = embeddings.embed_query("hello, world!")
vector[:5]

# embeddings.embed_query("Hello")


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001" ,
    google_api_key=gemini_api_key  # correct it
    )

vector = embeddings.embed_query("hello, world!")
vector[:10]


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966,
 -0.0018945280462503433,
 0.028477225452661514,
 -0.007562300190329552,
 0.011064725928008556,
 -0.005353901535272598]